In [ ]:
import plotly.express as px
import polars as pl
from functime.cross_validation import train_test_split
from functime.forecasting import linear_model
from functime.metrics import mase
from functime.preprocessing import scale
from functime.seasonality import add_calendar_effects, add_fourier_terms

pl.Config.set_tbl_width_chars(256)
pl.Config.set_fmt_str_lengths(256)

# The usernames are stored in the Details section of the MPQ. This can be used later to correlate "played_id"s with human friendly player name.
details_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/details.ipc")
details_df = (
    details_df
    .explode("player_list")
    .unnest("player_list")
    .filter(pl.col("name").str.ends_with("Sazed"))
    .select([pl.col("working_set_slot_id") + 1,pl.col("ext_fs_replay_sha256"),pl.col("ext_datetime")])
)

unit_born_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/unit_born.ipc")
unit_died_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/unit_died.ipc")
stats_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/stats.ipc")
init_data_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/init_data.ipc")
upgrades_df = pl.scan_ipc("/home/seb/git/s2protocol-rs/ipcs/upgrades.ipc")




latest_replay_digest = (
    details_df.collect()
    .sort("ext_datetime", descending=True)
    .select(pl.col("ext_fs_replay_sha256"))
    .limit(1)
)
# In this case I'm interested in my own user data.
my_username = "2-1-8459957-Sazed"
# To find your user, you can inspect a replay, for example the latest one, this is a combination of the blizzard ids, region, etc.
unit_born_df.filter((pl.col("ext_fs_replay_sha256") == latest_replay_digest)).group_by(
    "ext_replay_detail_player_name"
).agg(pl.count()).collect()

